In [1]:
pip list

Package                   Version
------------------------- --------------
absl-py                   2.1.0
alembic                   1.14.1
anyio                     4.8.0
archspec                  0.2.5
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asgiref                   3.8.1
asttokens                 3.0.0
astunparse                1.6.3
async-lru                 2.0.4
attrs                     25.1.0
babel                     2.17.0
beautifulsoup4            4.12.3
bleach                    6.2.0
boltons                   24.0.0
Brotli                    1.1.0
certifi                   2024.12.14
cffi                      1.17.1
chardet                   5.2.0
charset-normalizer        3.4.1
click                     8.1.8
cloudpickle               3.1.1
cmake                     3.31.4
colorama                  0.4.6
colorlog                  6.9.0
comm                      0.2.2
conda                     25.1.0
conda-build   

In [3]:
pip install djangorestframework celery redis pynmea2 pyserial channels python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.1
    Uninstalling tzdata-2025.1:
      Successfully uninstalled tzdata-2025.1

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

from typing import Dict, Any, Optional, Callable

import gymnasium as gym
from gymnasium.envs.registration import register, registry
from gymnasium.wrappers import RecordVideo
import time
import numpy as np

# from typing import Any, Dict
import torch
import torch.nn as nn

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

In [ ]:
# Create folders for logs and videos
log_folder = './marl_logs'
video_folder = os.path.join(log_folder, 'video')
os.makedirs(video_folder, exist_ok=True)
# PPO.load(os.path.join(log_folder, 'optuna/best_model/best_model.zip'), device='cpu').policy

In [ ]:
%matplotlib inline
%load_ext tensorboard

%tensorboard --logdir {log_folder} --host=0.0.0.0

In [ ]:
if 'MarineEnv-v0' not in registry:
    register(
        id='MarineEnv-v0',
        entry_point='marl:MarineEnv',  # String reference to the class
    )

def yield_random_seed():
    while True:
        yield np.random.randint(low=1, high=201)
seed_generator = yield_random_seed()

In [ ]:
env_kwargs = dict(
        render_mode='rgb_array',
        training_stage=2,
        timescale=1/3,
        training=False,
        total_targets=2,
    )
    
env = gym.make(
        'MarineEnv-v0',
        **env_kwargs,
    )

In [ ]:
print("Actual observation space:", (env.observation_space.shape))

In [ ]:
def run_video_rendering(
    agent: Any, 
    episodes: int = 3, 
    timescale: float = 1/6, 
    seed: Optional[int] = None, 
    record_video: bool = False, 
    episode_trigger: Optional[Callable[[int], bool]] = None, 
    name_prefix: Optional[str] = None, 
    video_folder: Optional[str] = None,
    marl: bool = False,
) -> None:
    """
    Runs a simulation of the MarineEnv-v0 environment using the given agent, with optional video recording.

    Parameters:
    - agent: The trained agent used for inference.
    - episodes (int): Number of episodes to run (default is 3).
    - timescale (float): The simulation timescale factor (default is 1/6).
    - seed (int, optional): Random seed for environment initialization. If None, a generated seed is used.
    - record_video (bool): If True, records the simulation as a video (default is False).
    - episode_trigger (function, optional): A function that determines which episodes get recorded.
    - name_prefix (str, optional): Prefix for recorded video file names.
    - video_folder (str, optional): Path to save recorded videos.
    - marl (bool): If True, running the env in MARL mode.

    Behavior:
    - If video recording is enabled, the environment is wrapped with RecordVideo.
    - Logs episode statistics, including total rewards, episode length, and termination status.
    - If recording, logs are saved to a text file in the specified video folder.
    - Displays simulation results in the console if video recording is disabled.

    Returns:
    - None. The function either logs the results to a file or prints them to the console.
    """
    
    if seed is None:
        seed = next(seed_generator)
        
    kwargs = dict(
        render_mode='rgb_array' if record_video else 'human',
        continuous=True,
        training_stage=2,
        timescale=timescale,
        training=False,
        total_targets=2,
        seed=seed,
    )
    
    env = gym.make(
        'MarineEnv-v0',
        **kwargs,
    )
    
    if record_video:
        from IPython.display import HTML
        from base64 import b64encode
        
        if episode_trigger is None:
            episode_trigger = lambda episode_id: True

        if video_folder is None:
            video_folder = video_folder   
        else:
            if name_prefix:
                video_folder = os.path.join(video_folder, name_prefix)
        
        # wrap environment for video recording
        env = RecordVideo(
            env=env, 
            video_folder=video_folder, 
            episode_trigger=lambda episode_id: True, 
            name_prefix=name_prefix)
        
    logged_episodes = []
    logged_rewards = []
    for episode in range(episodes):
        state, _ = env.reset()
        episode_rewards = 0

        if marl:
            eta = state[0][8]
        else:
            eta = state[8]
        
        for step in range(int(400 / timescale)):
            if marl:
                with torch.no_grad():
                    actions = [agent.predict(i, deterministic=True)[0] for i in state]
                next_state, reward, terminated, truncated, info = env.step(actions)
            else:
                with torch.no_grad():
                    action = agent.predict(state, deterministic=True)
            
                state, reward, terminated, truncated, info = env.step(action[0])
            
            if not record_video:
                env.render()
                
            episode_rewards += reward
            
            if terminated or truncated:
                break
                
            time.sleep(0.005)
         
        result_string = f'Episode: {episode}\nEpisode length: {step}, Elapsed real time: {round(step * timescale)} minutes, Initial WP ETA: {round(eta)} minutes\nEpisode total rewards: {episode_rewards :.2f}\nIs terminated: {info["terminated"]}, Is truncated: {info["truncated"]}\n============================\n'

        logged_episodes.append(result_string)
        logged_rewards.append(episode_rewards)

    evaluation = f'Mean: {np.array(logged_rewards).mean():.2f}, Std: {np.array(logged_rewards).std():.2f}, Initial seed: {seed}'
    logged_episodes.append(evaluation)
    # Open log file
    if record_video:
        log_file_path = os.path.join(video_folder, name_prefix + '.txt')
        with open(log_file_path, 'w') as log_file:
            log_file.write('\n'.join(logged_episodes))
            print(f"Training log saved at: {log_file_path}")
    else:
        print('\n'.join(logged_episodes))
    
    env.close()

In [ ]:
# setup of the training envs
train_env = make_vec_env(env_id='MarineEnv-v0', n_envs=8, env_kwargs=env_kwargs)
eval_env = gym.make('MarineEnv-v0', **env_kwargs)
eval_env = Monitor(eval_env)

In [ ]:
# setup of the defaut kwargs for initial training of PPO agent
default_kwargs = {
    'learning_rate': 3e-4,
    'n_steps': 2048,
    'batch_size': 64,
    'n_epochs': 10,
    'gamma': 0.99, 
    'gae_lambda': 0.95,
    'clip_range': 0.2,
    'ent_coef': 0.0,
    'vf_coef': 0.5, 
    'max_grad_norm': 0.5,
    'target_kl': None,
    'tensorboard_log': log_folder,
}

In [ ]:
# establish initial agent
agent = PPO(
        policy='MlpPolicy',
        env=train_env,
        verbose=0,
        device='cpu',
        **default_kwargs,
    )

In [ ]:
# eval callback - used to track the statistics of the training
eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=os.path.join(log_folder, 'default/best_model/'),
        log_path=os.path.join(log_folder, 'default/results/'),
        eval_freq=5000,
        deterministic=True,
        render=False
    )

In [ ]:
# env.observation_space

In [ ]:
# agent.observation_space

In [ ]:
agent.learn(total_timesteps=int(1e5), reset_num_timesteps=True, tb_log_name='default/default_best', progress_bar=True, callback=eval_callback)

In [ ]:
env.reset()
agent = PPO.load('marl_logs/default/best_model/best_model.zip')

In [ ]:
run_video_rendering(agent, episodes=1)

In [ ]:
run_video_rendering(agent, episodes=1, marl=False, seed=42)

In [ ]:
env = agent.env

In [ ]:
raw_env = env.envs[5]

In [ ]:
env = raw_env
while hasattr(env, 'env'):
    env = env.env

In [ ]:
env.reset()

In [ ]:
obs, reward, terminated, truncated, info = env.step(env.action_space.sample())
print("Step obs:", obs)



In [ ]:
info
